In [16]:
import pymysql
import os

In [2]:
HOST = "rc1a-2f6hdfabdf89l29u.mdb.yandexcloud.net"
PORT = 3306
USER = "admin"
PASSWORD = "qwerty"
DATABASE = "database"

ssl_ca = os.path.expanduser("mysql\root.crt")

In [3]:
try:
    conn = pymysql.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database=DATABASE,
        ssl={'ca': ssl_ca},
        charset='utf8mb4',
        autocommit=True
    )
    cur = conn.cursor()
    cur.execute("SELECT VERSION(), DATABASE();")
    version, db = cur.fetchone()
    print(f"Версия MySQL: {version}")
    print(f"Текущая БД: {db}")
except Exception as e:
    print(f"{type(e).__name__}: {e}")

FileNotFoundError: [Errno 2] No such file or directory


# 1. Создаём новую БД

In [5]:

new_db_name = "database2"

cur.execute(f"USE `{new_db_name}`;")
print(f"USE `{new_db_name}`")

cur.execute("SELECT VERSION(), DATABASE();")
version, db = cur.fetchone()
print(f"Версия MySQL: {version}")
print(f"Текущая БД: {db}")

USE `database2`
Версия MySQL: 8.0.41-32
Текущая БД: database2


2. Создание таблицы

In [6]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS test (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        age INT
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
""")
print("таблица test создана")

таблица test создана


Вставляем данные

In [9]:
data = [
    ("Человек 1", 18),
    ("Человек 2", 35),
    ("Человек 3", 25),
    ("Человек 4", 27),
    ("Человек 5", 32)
]

cur.executemany("""
    INSERT INTO characters (name, role, appearance_count)
    VALUES (%s, %s, %s);
""", data)

5

Работа с DataFrame через sqlalchemy

In [17]:
import pandas as pd

df = pd.read_csv('top10_words.csv')
print(df)

  Слово  Частота
0    не      634
1   что      559
2    ты      351
3    то      306
4    он      242
5  если      184
6    же      181
7   как      174
8    на      173
9    от      138


In [13]:
from sqlalchemy import create_engine

engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{new_db_name}",
    connect_args={"ssl": {"ca": ssl_ca}}
)

df.to_sql('top_words', con=engine, if_exists='replace', index=False)

10

In [20]:
df = pd.read_sql("SELECT * FROM characters;", engine)
df

,id,name,age
0,1,Человек 1,18
1,2,Человек 2,35
2,3,Человек 3,25
3,4,Человек 4,27
4,5,Человек 5,32


In [21]:
topDf = pd.read_sql("SELECT * FROM top_words;", engine)
topDf

,Слово,Частота
0,не,634
1,что,559
2,ты,351
3,то,306
4,он,242
5,если,184
6,же,181
7,как,174
8,на,173
9,от,138
